<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Setting-Up-Beutiful-Soup" data-toc-modified-id="Setting-Up-Beutiful-Soup-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Setting Up Beutiful Soup</a></span></li><li><span><a href="#Inspecting-HTML" data-toc-modified-id="Inspecting-HTML-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Inspecting HTML</a></span></li><li><span><a href="#Making-DF" data-toc-modified-id="Making-DF-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Making DF</a></span></li></ul></li><li><span><a href="#Printing-Shape-as-requested" data-toc-modified-id="Printing-Shape-as-requested-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Printing Shape as requested</a></span><ul class="toc-item"><li><span><a href="#Saving-DF" data-toc-modified-id="Saving-DF-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Saving DF</a></span></li><li><span><a href="#----------------------------------------------Checkpoint-----------------------------" data-toc-modified-id="----------------------------------------------Checkpoint------------------------------1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>----------------------------------------------Checkpoint-----------------------------</a></span></li></ul></li></ul></div>

In [3]:
import pandas as pd
import numpy as np

# Scraping Data from Wiki

#### Setting Up Beutiful Soup

In [4]:
from bs4 import BeautifulSoup
import urllib3

url="https://en.wikipedia.org/wiki/List_of_city-designated_neighbourhoods_in_Toronto"
url="https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
http=urllib3.PoolManager()
html_doc=http.request('GET', url).data
soup=BeautifulSoup(html_doc, 'html.parser')

/home/boo/anaconda3/envs/python/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


#### Inspecting HTML

In [5]:
second_table=soup.findAll("table")[0]
second_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [6]:
for row in second_table.findAll("tr")[1:]:
    print("---------------------Row---------------")
    print(row.text.strip())
    print("---------------------------------------\n\n")

---------------------Row---------------
M1A
Not assigned
Not assigned
---------------------------------------


---------------------Row---------------
M2A
Not assigned
Not assigned
---------------------------------------


---------------------Row---------------
M3A
North York
Parkwoods
---------------------------------------


---------------------Row---------------
M4A
North York
Victoria Village
---------------------------------------


---------------------Row---------------
M5A
Downtown Toronto
Harbourfront
---------------------------------------


---------------------Row---------------
M5A
Downtown Toronto
Regent Park
---------------------------------------


---------------------Row---------------
M6A
North York
Lawrence Heights
---------------------------------------


---------------------Row---------------
M6A
North York
Lawrence Manor
---------------------------------------


---------------------Row---------------
M7A
Queen's Park
Not assigned
----------------------------

In [7]:
first_row=second_table.findAll("tr")[0]
print(first_row.text.strip())

Postcode
Borough
Neighbourhood


#### Making DF

In [8]:
neighbourhood_list=[]
boroughs_list=[]
postcode_list=[]

"""
It will be used for another purpose 
boroughs_abv={
            "OCoT":"Old City of Toronto",
            "S":"Scarborough",
            "NY":"North York",
            "E":"Etobicoke", 
            "Y":"York", 
            "EY":"East York", 
            }
"""


rows=second_table.findAll("tr")
for row in rows[1:]:
    cells=row.findAll("td")
    
    borough=cells[1].text.strip()
    
    if "not assigned" not in borough.lower(): 
        neighbourhood=cells[2].text.strip()
        postcode=cells[0].text.strip()
        
        if "not assigned" in neighbourhood.lower():
            neighbourhood=borough
        
        if postcode in postcode_list:
            i=postcode_list.index(postcode)
            neighbourhood_list[i]=neighbourhood_list[i]+", "+neighbourhood
    
        else:
            neighbourhood_list.append(neighbourhood)
            boroughs_list.append(borough)
            postcode_list.append(postcode)

toronto_dict={
                "PostalCode":postcode_list,
                "Borough":boroughs_list,
                "Neighbourhood":neighbourhood_list,
            }

tt_df=pd.DataFrame(toronto_dict)
tt_df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


### Printing Shape as requested

In [9]:
tt_df.shape

(103, 3)

####  Saving DF

In [11]:
tt_df.to_csv("toronto_data.csv", index=False)

#### ----------------------------------------------Checkpoint-----------------------------